### Meta Labeling II

Meta Labeling은 Marcos Lopez de Prado(2018)가 저서 Advances in Financial Machine Learning에서 제안한 모형으로, labeling문제에 직면한 금융 머신러닝 프로젝트에 적절한 방법을 제시한다. 다음과 같은 논리로 이뤄진다

1. Feature 준비 : 자산에 대한 가격 데이터, technical feature등을 포함한 feature matrix를 준비한다
2. Primary Model : 자산의 가격이 상승할지, 하강할지 판단하는 모형이다. 여기에는 자신만의 판단 기준을 내세울 수 있으며, 때로는 애널리스트의 판단이 될 수 있다
3. Secondary Model : Primary Model에서 나온 정보를 가지고 거래를 집행할지, 하지 않을지를 판단하는 모형이다. 거래 집행 유무만 판단하므로 이진 분류 문제가 된다

오늘은 13주차에 이어서 technical feature를 이용해 간단한 이동평균 매수매도 전략을 Primary Model로 제작한 후, Ensemble Model을 활용하여 Secondary Model을 제작해 매매 의사결정 모형을 만드는 것을 목표로 한다

#### 1. Data Import

아래의 library를 활용해 Apple의 가격 시계열을 불러오자. 한국은 공매도 집행이 금지되었으므로 고려하지 않는다. 기간은 2000년 1월 1일부터 12월 22일까지로 지정한다. 시계열이므로 정상성을 위해 로그 차분을 진행한다

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

start_date = '2000-01-01'
end_date = '2023-12-23'

# write your code here!

#### 2. Fractionally Differentiated Features

Prado(2018)는 저서 Advances in Financial Machine Learning을 통해 시장의 장기 기억을 담고 있는 시계열의 경우 Machine Learning 학습에 도움이 된다고 주장하였다

`IntegratedSeries.py`의 함수를 이용하여 원 시계열 데이터에 0.6차분을 수행하라. 사용 방법은 7주차 내용을 참고하라

In [ ]:
import IntegratedSeries as IS
import statsmodels.api as sm

# write your code here!

#### 3. Technical Features

`ta` library에는 다양한 기술적 분석들에 사용 가능한 feature를 제작해 준다. ta library를 활용하여 다음과 같은 feature를 만들어 주도록 한다

- money flow index : window = 20
- on balance volume : window = 20
- rsi : window = 20
- bollinger band : window = 20, window_dev = 1
- volume weighted average price : window = 20
- tsi indicator : window_fast = 13, window_slow = 25

In [ ]:
from ta.volatility import BollingerBands
from ta.volume import money_flow_index, on_balance_volume
from ta.momentum import rsi
from ta.volume import volume_weighted_average_price
from ta.momentum import tsi

#indicator_bb = BollingerBands(close = apple["Close"], window = 20, window_dev = 1)

#apple['bb_bbm'] = indicator_bb.bollinger_mavg()
#apple['bb_bbh'] = indicator_bb.bollinger_hband()
#apple['bb_bbl'] = indicator_bb.bollinger_lband()

#apple['mfi'] = money_flow_index(high = apple.High, 
#                                low = apple.Low, 
#                                close = apple.Close, 
#                                volume = apple.Volume,
#                                window = 20)
#apple['obv'] = on_balance_volume(close = apple.Close, 
#                                   volume = apple.Volume)
#apple['rsi'] = rsi(close = apple.Close, window = 21)
#apple['return'] = apple_ret
#apple['vwap'] = volume_weighted_average_price(high = apple['High'],
#                                              low = apple['Low'],
#                                              close = apple['Close'],
#                                              volume = apple['Volume'],
#                                              window = 20)
#apple['tsi'] = tsi(close = apple['Close'],
#                   window_fast = 13,
#                   window_slow = 15)

#### 4. Corwin Schultz Spread

10주차의 `MicroStructure.py`를 이용하여 corwin schultz spread를 계산하라. 사용방법은 10주차의 내용을 참고하도록 한다

In [ ]:
import MicroStructure as ms

# write your code here!

#### 5. Primary Modeling

Primary Model에서는 방향만을 추정한다. 여기서는 Bollinger Band전략을 사용하여 가격이 상승할 것인지, 하락할 것인지 labeling을 하자. 여기서 상승은 1, 하락은 -1, 횡보상태는 0으로 labeling한다

In [ ]:
# write your code here!

#### 6. Meta Labeling

Secondary Model은 Primary Model에서 나온 방향을 이용해 거래의 실행 여부만을 평가한다. 3개의 레이블[-1, 0, 1]을 이진 분류 모형[0, 1]으로 다시 labeling을 진행한다. 이 과정을 Meta Labeling이라고 한다

In [ ]:
# write your code here!

#### 7. Secondary Modeling

Meta Label을 target으로 지정하고, 2번에서 지정한 technical feature들을 이용하여 이진 분류 모형을 수행하라. 즉, feature matrix에는 아래와 같은 데이터들이 들어간다

- fractionally differenctiated features
- money flow index
- on balance volume
- RSI
- TSI
- VWAP
- log return
- corwin schultz spread
- meta label (target)

여기서, Bollinger Band 관련 feature들은 labeling에 사용되었으므로 고려하지 않는다

모형의 검증을 위해 test data는 0.2로 지정하고 shuffle = False로 지정한다

In [ ]:
from sklearn.model_selection import train_test_split

# write your code here!

**7.1 RandomForest Classifier**

Secondary Model로 Random Forest Classifier를 사용하고 성능을 평가하라. Hyper Parameter는 다음과 같이 지정한다

```
n_estimators = 1000
sample_weights = 'balanced_subsample'
criterion = 'entropy'
ccp_alpha = 0.001
random_state = 42
```

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# write your code here!

**7.2 Ada Boosting**

동일한 작업을 AdaBoosting을 통해 확인하라. 단, 기초 모형은 Decision Tree를 사용하고, Hyper Parameter는 다음과 같이 지정하도록 한다

**Estimator : Tree**
```
    max_depth = 1,
    criterion = 'entropy',
    class_weight = 'balanced',
    random_state = 42
```

**Adaboost**
```
    n_estimators = 1000,
    random_state = 42
```

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# write your code here!

**7.3 Gradient Boosting**

주어진 문제를 Gradient Boosting을 활용해 추정하자. Hyper Parameter는 아래와 같이 지정한다

```
    n_estimators = 1000, 
    learning_rate = 1.0, 
    max_depth = 1, 
    random_state = 42
```

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# write your code here!

**7.4 CatBoosting**

주어진 문제를 CatBoost를 활용해 예측해 보자. 아래의 Hyper Parameter를 사용하고, Sample Weights는 Random Forest에서 사용했던 balanced_subsample과 동일한 기능을 하도록 조정한다

```
    class_weights = class_weights,
    n_estimators = 1000,
    loss_function = 'Logloss',
    random_state = 42,
    learning_rate = 0.1,
    verbose = 0
```

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
from catboost import CatBoostClassifier

# write your code here!

#### 8. Compare Results

추정기간의 성능을 비교해 보자. 매매 기회를 최대한 많이 잡아내기 위해서는(실제 1 중에서 최대한 많이 잡아내는 것) 어떤 모형을 쓰는 것이 가장 효율적인가?